In [75]:
########################################################################################
## PACKAGE IMPORTING AND GLOBAL VARIABLE DEFINITION ########
########################################################################################

# v080, using continous fake data to give more physcial ressults
# from 721, set bias=False for LSTM and MLP; LSMT layer =1 and no dropout # one layer LSTM may be enough
# V0.7  Update: Use large hidden size; MSE as loss function; Based on V0.60
# V0.6  Update: Type 3 has better performance than type 4
# V0.51 Update: to run on the oscilloscope data
# V0.40 Update: to combine different networks
# V0.31 Update: tof changed to tensor format, which is a grammar improvement instead of bug fix
# V0.32 Update: Use L2 regularization instead of drapout

from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
from torch import optim
import torch    # for setting seed to make it reproducible
import torch.nn as nn
import matplotlib.pyplot as plt  # for plotting
import matplotlib.mlab as mlab  # for plotting
import struct  # for binary file reading
import numpy as np
import time  # for time recording
import math  # for sqrt
from pathlib import Path  # for the size of the binary file
from scipy.stats import norm  # for gaussian fit
import sys  # for parameter receiving
import os  # for mkdir
from progressbar import *

print("all packages successfully imported")

all packages successfully imported


In [76]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

time_epoch = time.time()

SEED=5  # reproducible
DATA_VERSION = '0710_5'


torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True
torch.manual_seed(SEED)
SAMPLE_EPOCH = 1  # test every xx epochs to record mu, timeresolution, and loss

EPOCHS = 3
MODEL_VERSION = '800'

WAVE_LENGTH = 120
NCHANNEL = 8

LR = 0.001 + 0.001*np.random.rand()
LRD = 0.9 + 0.1*np.random.rand()
HIDDEN_SIZE = 60 + int(10*np.random.rand())
NUM_LSTM_LAYERS = 4 #+int(2*np.random.rand())
BATCH_SIZE = 256
DROP_OUT = 0.3
WEIGHT_DECAY = 0.0004 + 0.0004*np.random.rand()  # lambda of L2 regularization

print("all global vars generated")

all global vars generated


In [80]:
########################################################################################
## CLASS AND FUNCTIION DEFINITION ########
########################################################################################


class Waveforms(Dataset):

    def __init__(self, root_dir, testRate = 0.3, train=True):
        self.root_dir = root_dir
        self.train = train
        self.file = open(root_dir, 'rb')
        self.waveLength = WAVE_LENGTH
        self.NChannel = NCHANNEL
        self.NEvents = Path(self.root_dir).stat().st_size / \
            (self.waveLength*self.NChannel+1)/4
        print('NEvents = ', self.NEvents)
        self.testRaio = testRate

    def __len__(self):
        if self.train:
            return int(self.NEvents*(1.0-self.testRaio))
        else:
            return int(self.NEvents*self.testRaio)

    def __getitem__(self, index):
        if self.train:
            self.file.seek((index+int(self.NEvents*self.testRaio))
                           * (self.NChannel*self.waveLength+1)*4, 0)
        else:
            self.file.seek((index)*(self.NChannel*self.waveLength+1)*4, 0)
        data = self.file.read(self.waveLength*self.NChannel*4)
        wave = struct.unpack(str(self.NChannel*self.waveLength)+'f', data)
        wave_tensor = torch.tensor(wave, dtype=torch.float32).reshape(
            self.NChannel, self.waveLength)#.t()
        data = self.file.read(4)
        tof = struct.unpack('f', data)
        tof_tensor = torch.tensor(tof, dtype=torch.float32)
        sample = {'waveform': wave_tensor, 'tof': tof_tensor}
        return sample


class MLP(nn.Module):
    def __init__(self, input_size, common_size):
        super(MLP, self).__init__()
        self.linear = nn.Sequential(
            nn.Linear(input_size, input_size // 2),
            nn.ReLU(inplace=True),
            nn.Linear(input_size // 2, input_size // 4),
            nn.ReLU(inplace=True),
            nn.Linear(input_size // 4, common_size)
        )

    def forward(self, x):
        out = self.linear(x)
        return out


class ComNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=NCHANNEL+1,
            hidden_size=HIDDEN_SIZE,
            num_layers=NUM_LSTM_LAYERS,
            batch_first=True,
            bias=False,
            dropout = DROP_OUT
        )
        self.out1 = nn.Linear(HIDDEN_SIZE, 1, bias=False)
        self.magnifier1 = MLP(NCHANNEL*WAVE_LENGTH, WAVE_LENGTH) # N to one


    def forward(self, x):
        x.dim
        extra1 = self.magnifier1(x.view(x.size(0),NCHANNEL*WAVE_LENGTH))
        extra1 = extra1.unsqueeze(2)
        x = torch.cat((x,extra1), 2)

        lstm_out, (h_n, h_c) = self.lstm(x, None)
        # lstm_out[:, -1, :] is the last lines of the input batches, since it's a many to one IO structure.
        out = (self.out1(lstm_out[:, -1, :]))
        return out


class CNN_Net(nn.Module):
    def __init__(self):
        super(CNN_Net, self).__init__()
        self.conv1 = nn.Conv1d( 8 ,256 ,120)
        self.pool = nn.MaxPool2d(2, 2)
        #self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(256*256, 1)
        #self.fc2 = nn.Linear(120, 84)
        #self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x)
        #x = self.pool(F.relu(self.conv2(x)))
        #x = x.view(-1, 16 * 5 * 5)
        #x = self.fc1(x)
        #x = F.relu(self.fc2(x))
        #x = self.fc3(x)
        return x



def testDataset(network, dataVersion, outputPrefix, input_prefix = "./Codes/V0.70/Data/"):

    inputFileName = input_prefix+dataVersion+".bin"
    testWaveformDataset = Waveforms(inputFileName, testRate = 1.0, train=False)
    testData = DataLoader(testWaveformDataset,
                          batch_size=BATCH_SIZE, shuffle=False)

    residualTofList = []
    predictedTofList = []
    labelTofList = []
    print("Test Data: " + dataVersion)
    mu=sigma=0
    with torch.no_grad( ):
        for index, batch_data in enumerate(testData):
            input = Variable(batch_data['waveform'])#.cuda()
            b_y = Variable(batch_data['tof']).squeeze()#.cuda().squeeze()
            output = comNN(input).squeeze()
            residualTofList.extend(
                (output-b_y).cpu().detach().numpy().tolist())
            predictedTofList.extend(output.cpu().detach().numpy().tolist())
            labelTofList.extend(b_y.cpu().detach().numpy().tolist())

    (mu, sigma) = norm.fit(residualTofList)
    n, bins, patches = plt.hist(residualTofList, 100, range=(-1, 1), density=2)
    # add a 'best fit' line
    y = norm.pdf(bins, mu, sigma)
    l = plt.plot(bins, y, 'r--', linewidth=2)
    plt.xlabel('Tof residual [ns]')
    plt.title(
        r'$\mathrm{Tof\ residual:}\ \mu=%.3f ns,\ \sigma/\sqrt{2}=%.3f ns$' % (mu, sigma/math.sqrt(2)))
    plt.grid(True)
    plt.savefig(outputPath + outputPrefix +
                "Test_" + dataVersion + ".png")
    plt.clf()


def my_mse_loss(x, y, n): 
    return torch.mean(torch.pow(torch.abs(x - y), n))

print("all classes defined")

all classes defined


In [78]:
########################################################################################
## INPUT/OUTPUT PATH SETTING ########
########################################################################################
workspace = 'C:/Users/x/OneDrive/CODING/'
print('workspace:',workspace,os.path.exists(workspace))
fileName = workspace+"/data/"+DATA_VERSION+".bin"
outputPath = workspace+DATA_VERSION+"_"+MODEL_VERSION+"_Aug20th_v1/"
if os.path.exists(outputPath) == False:
    os.mkdir(outputPath)
outputPrefix = DATA_VERSION+"_SEED"+str(SEED).zfill(4) + "_E"+str(EPOCHS)

if True:
  print('workspace:',workspace,os.path.exists(workspace))
  print('fileName:',fileName,os.path.exists(fileName))
  print('outputPath:',outputPath,os.path.exists(outputPath))

workspace: C:/Users/x/OneDrive/CODING/ True
workspace: C:/Users/x/OneDrive/CODING/ True
fileName: C:/Users/x/OneDrive/CODING//data/0710_5.bin True
outputPath: C:/Users/x/OneDrive/CODING/0710_5_800_Aug20th_v1/ True


In [81]:
########################################################################################
## EXCUTION: Training ########
########################################################################################


trainWaveformDataset = Waveforms(fileName, train=True)
testWaveformDataset = Waveforms(fileName, train=False)
trainData = DataLoader(trainWaveformDataset, batch_size=BATCH_SIZE, shuffle=True)
testData = DataLoader(testWaveformDataset, batch_size=BATCH_SIZE, shuffle=False)

comNN = CNN_Net()

use_gpu = torch.cuda.is_available()  # 判断是否有GPU加速
if use_gpu:
    comNN = comNN.cuda()
    

print(comNN)

optimizer = optim.Adam(comNN.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
# https://pytorch.org/docs/stable/optim.html#torch.optim.lr_scheduler.ReduceLROnPlateau
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=LRD)
print(optimizer)
# loss_func = nn.MSELoss().cuda()
loss_func = my_mse_loss

testLoss_his = []
trainLoss_his = []
timeResolution_his = []
mu_his = []
residualTofList = []
predictedTofList = []
labelTofList = []

print("Data: " + DATA_VERSION + "; SEED = " + str(SEED)+"； Epochs = " + str(EPOCHS))
progress = ProgressBar()
for epoch in progress(range(EPOCHS)):
    epoch_loss = 0  # for LR decay rate scheduler
    for index, batch_data in enumerate(trainData):
        input = Variable(batch_data['waveform']).squeeze()#.cuda()
        b_y = Variable(batch_data['tof']).squeeze()#.cuda().squeeze()
        # print(input.size())
        output = comNN(input).squeeze()
        print(len(output),len(b_y),"1")
        loss = loss_func(output, b_y, 2)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_loss += loss.data.cpu().numpy()
    scheduler.step(epoch_loss)

    # to store the sigma, mu, loss
    if (epoch) % SAMPLE_EPOCH == 0:
        trainLoss_his.append(loss.data.cpu().numpy())
        residualTofList.clear()
        predictedTofList.clear()
        labelTofList.clear()
        for index, batch_data in enumerate(testData):
            input = Variable(batch_data['waveform'])#.cuda().cuda()
            b_y = Variable(batch_data['tof']).squeeze()#.cuda().cuda().squeeze()
            output = comNN(input).squeeze()
            loss = loss_func(output, b_y, 2)
            residualTofList.extend(
                (output-b_y).cpu().detach().numpy().tolist())
            predictedTofList.extend(output.cpu().detach().numpy().tolist())
            labelTofList.extend(b_y.cpu().detach().numpy().tolist())
            # print('output length: ',output.cpu().detach().numpy().tolist().__len__())
        # print('toflist length: ',tofList.__len__())
        (mu, sigma) = norm.fit(residualTofList)
        timeResolution_his.append(sigma/math.sqrt(2))
        mu_his.append(mu)
        testLoss_his.append(loss.data.cpu().numpy())


# save model
torch.save(comNN, outputPath+outputPrefix)

N/A% (0 of 3) |                          | Elapsed Time: 0:00:00 ETA:  --:--:--

NEvents =  10579.0
NEvents =  10579.0
CNN_Net(
  (conv1): Conv1d(8, 256, kernel_size=(120,), stride=(1,))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=65536, out_features=1, bias=True)
)
Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.0012219931710897396
    weight_decay: 0.0007674443631751687
)
Data: 0710_5; SEED = 5； Epochs = 3
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
256 256 1
237 237 1


RuntimeError: The size of tensor a (256) must match the size of tensor b (237) at non-singleton dimension 1

In [ ]:
########################################################################################
## EXCUTION:Plotting ########
########################################################################################
time_start = time.time()
plt.plot(testLoss_his, label='testLoss')
plt.plot(trainLoss_his, label='trainLoss')
plt.grid(True)
# plt.ylim([0.01, 0.08])
# plt.ylabel("MSE [ns^2]")
plt.title("Loss/MSE Vs Epochs")
plt.xlabel("sampled every "+str(SAMPLE_EPOCH)+" epochs")
plt.legend()
plt.savefig(outputPath+outputPrefix+"_Loss.png")
plt.clf()

plt.plot(mu_his)
plt.grid(True)
plt.title("Average predicted ToF Vs Epochs")
plt.ylabel("Average of predicted ToF [ns]")
plt.ylim([-0.1, 0.1])
plt.xlabel("sampled every "+str(SAMPLE_EPOCH)+" epochs")
plt.savefig(outputPath+outputPrefix+"_Mu.png")
plt.clf()

plt.plot(timeResolution_his)
plt.ylabel("Time Resolution [ns]")
plt.grid(True)
plt.ylim([0.05, 0.30])
plt.title("TimeResolution Vs Epochs")
plt.xlabel("sampled every "+str(SAMPLE_EPOCH)+" epochs")
plt.savefig(outputPath+outputPrefix+"_TR.png")
plt.clf()

(mu, sigma) = norm.fit(residualTofList)
n, bins, patches = plt.hist(residualTofList, 100, range=(-1, 1), density=1)
# add a 'best fit' line
y = norm.pdf(bins, mu, sigma)
l = plt.plot(bins, y, 'r--', linewidth=2)
plt.xlabel('Tof residual [ns]')
# plt.ylabel('Probability')
plt.title(r'$\mathrm{Tof\ residual:}\ \mu=%.3f ns,\ \sigma/\sqrt{2}=%.3f ns$' %(mu, sigma/math.sqrt(2)))
plt.grid(True)
plt.savefig(outputPath+outputPrefix+"_TofHist.png")
plt.clf()